In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import torch
torch.set_default_dtype(torch.float64)


In [2]:
n_qubits = 2
noise_type = "depolarizing"  # Change to "bitflip", "phaseflip", "amplitude_damping"
noise_strength = 0.05   

In [3]:
iris = datasets.load_iris()
X = iris.data[:, :n_qubits]  # only first n_qubits features
y = iris.target

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to tensor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float64)
y_train = torch.tensor(y_train, dtype=torch.float64)
X_test = torch.tensor(X_test, dtype=torch.float64)
y_test = torch.tensor(y_test, dtype=torch.float64)

In [4]:
# -----------------
# DEVICE WITH NOISE
# -----------------
def get_noise_model(dev):
    """Attach noise gates according to noise_type"""
    if noise_type == "depolarizing":
        def noise():
            for w in range(n_qubits):
                qml.DepolarizingChannel(noise_strength, wires=w)
    elif noise_type == "bitflip":
        def noise():
            for w in range(n_qubits):
                qml.BitFlip(noise_strength, wires=w)
    elif noise_type == "phaseflip":
        def noise():
            for w in range(n_qubits):
                qml.PhaseFlip(noise_strength, wires=w)
    elif noise_type == "amplitude_damping":
        def noise():
            for w in range(n_qubits):
                qml.AmplitudeDamping(noise_strength, wires=w)
    else:
        def noise():
            pass
    return noise

dev = qml.device("default.mixed", wires=n_qubits)
noise_layer = get_noise_model(dev)

# -----------------
# QNODE
# -----------------
@qml.qnode(dev, interface="torch", diff_method="backprop")
def circuit(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    noise_layer()
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    noise_layer()
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# -----------------
# TORCH LAYER
# -----------------
weight_shapes = {"weights": (1, n_qubits, 3)}
qlayer = qml.qnn.TorchLayer(circuit, weight_shapes)

class HQNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.q = qlayer
        self.fc = nn.Linear(n_qubits, 1)

    def forward(self, x):
        return self.fc(self.q(x))

model = HQNN()
model = model.to(dtype=torch.float64)

In [8]:
# -----------------
# TRAINING
# -----------------
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

epochs = 30
for epoch in range(epochs):
    optimizer.zero_grad()
    preds = model(X_train)
    loss = loss_fn(preds.squeeze(), y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

# -----------------
# EVALUATION
# -----------------
with torch.no_grad():
    start_time = time.time()
    preds = model(X_test).detach().cpu().numpy()
    end_time = time.time()

mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)


Epoch 5/30, Loss: 0.452506
Epoch 10/30, Loss: 0.399437
Epoch 15/30, Loss: 0.364959
Epoch 20/30, Loss: 0.345536
Epoch 25/30, Loss: 0.336096
Epoch 30/30, Loss: 0.331728


In [9]:

print("\n--- Evaluation Metrics ---")
print(f"Noise type       : {noise_type}")
print(f"Noise strength   : {noise_strength}")
print(f"Number of qubits : {n_qubits}")
print(f"MSE              : {mse:.6f}")
print(f"RMSE             : {rmse:.6f}")
print(f"MAE              : {mae:.6f}")
print(f"R²               : {r2:.6f}")
print(f"epochs           : {epochs}")
print(f"Inference time   : {(end_time - start_time):.6f} seconds")


--- Evaluation Metrics ---
Noise type       : depolarizing
Noise strength   : 0.05
Number of qubits : 2
MSE              : 0.277537
RMSE             : 0.526818
MAE              : 0.404638
R²               : 0.602888
epochs           : 30
Inference time   : 0.006981 seconds
